# <center> <font color=Blue> Computer Vision </font> </center>
## <center> Face Detection Using HOG </center>


## S.Alireza Mousavizade

***

<p style="font-family: 'Vazir'; text-align: right; direction: rtl">
    در این تمرین مساله
    <span style="direction: rtl">Face Detection</span>
    با همان داده های ورودی را این بار به کمک
    <span style="direction: rtl">Neural Network</span>
    حل میکنیم.
</p>

***

## Dataset:

### Positive Images Dataset :
####  <a href="http://vis-www.cs.umass.edu/lfw">LFW Dataset</a>

### Negative Images Dataset :
####  <a href="http://www.vision.caltech.edu/Image_Datasets/Caltech256/">Caltech 256 Data Set</a>
<p style="font-family: 'Vazir Thin'; text-align: right; direction: rtl">
    (تصاویر حاوی چهره از این دیتاست حذف شده است.)
</p>

<hr style="height:3px;border:none;color:#333;background-color:#333;" />

## Algorithm:

### 0- Instantiate <code style="">&#x200E;HOGFaceDetector&#x200E;</code> Object:

<p style="font-family: 'Vazir'; text-align: right; direction: rtl">
    تابع <code style="">&#x200E;__init__&#x200E;</code> (سازنده این کلاس) این عمل را انجام میدهد:
</p>

In [1]:
def __init__(self,
         positive_train_dir_path,
         negative_train_dir_path,
         winSize,
         train_set_path,
         validation_set_path,
         test_set_path,
         train_hog_features_path,
         validation_hog_features_path,
         test_hog_features_path,
         svm_model_path,
         y_predict_path):

self.positive_train_dir_path = positive_train_dir_path
self.negative_train_dir_path = negative_train_dir_path

self.X_train, self.Y_train = None, None
self.X_validation, self.Y_validation = None, None
self.X_test, self.Y_test = None, None

self.hog_object = None
self.winSize = winSize

self.train_HOG_features = None
self.validation_HOG_features = None
self.test_HOG_features = None

self.Y_test_predicted = None

self.scalar = None
self.svm_kernel = ''

self.classifier_model = None

self.average_precision_recall_score = 0

self.train_set_path = train_set_path
self.validation_set_path = validation_set_path
self.test_set_path = test_set_path
self.train_hog_features_path = train_hog_features_path
self.validation_hog_features_path = validation_hog_features_path
self.test_hog_features_path = test_hog_features_path
self.svm_model_path = svm_model_path
self.y_predict_path = y_predict_path

IndentationError: expected an indented block (<ipython-input-1-3aba468b91c8>, line 14)

***

### 1- Read Images & Separate to Train-Validation-Test Data
<p style="font-family: 'Vazir'; text-align: right; direction: rtl">
    تابع <code style="">&#x200E;initialize&#x200E;</code>  از شی HOGFaceDetector این عمل را انجام میدهد:



In [ ]:
def initialize(self):
    # p: Positive / n: Negative

    retriever1 = Retriever(self.train_set_path)
    retriever2 = Retriever(self.validation_set_path)
    retriever3 = Retriever(self.test_set_path)

    try:
        self.X_train, self.Y_train = retriever1.retrieve()
        self.X_validation, self.Y_validation = retriever2.retrieve()
        self.X_test, self.Y_test = retriever3.retrieve()

    except:
        p_X, p_Y, positive_patches_size = self.retrieve_samples_of(self.positive_train_dir_path,
                                                                   label=1)

        n_X, n_Y, negative_patches_size = self.retrieve_samples_of(self.negative_train_dir_path,
                                                                   label=0)

        train_size = 20000 // 2
        validation_size = 2000 // 2
        test_size = 2000 // 2
        # total_size = train_size + validation_size + test_size

        p_X_train, p_Y_train, \
        p_X_validation, p_Y_validation, \
        p_X_test, p_Y_test = HOGFaceDetector.train_validation_test_split(p_X, p_Y,
                                                                         train_size,
                                                                         validation_size,
                                                                         test_size)

        n_X_train, n_Y_train, \
        n_X_validation, n_Y_validation, \
        n_X_test, n_Y_test = HOGFaceDetector.train_validation_test_split(n_X, n_Y,
                                                                         train_size,
                                                                         validation_size,
                                                                         test_size)

        self.X_train, self.Y_train = p_X_train + n_X_train, p_Y_train + n_Y_train
        retriever1.save([self.X_train, self.Y_train])

        self.X_validation, self.Y_validation = p_X_validation + n_X_validation, p_Y_validation + n_Y_validation
        retriever2.save([self.X_validation, self.Y_validation])

        self.X_test, self.Y_test = p_X_test + n_X_test, p_Y_test + n_Y_test
        retriever3.save([self.X_test, self.Y_test])

    logging.info('HOGFaceDetector object initialized ... ')

    return self

***

### 2- HOG Feature Vector Extraction

<p style="font-family: 'Vazir'; text-align: right; direction: rtl">
    تابع  <code style="">&#x200E;extract_features&#x200E;</code> از کلاس  <code style="">&#x200E;HOGFaceDetector&#x200E;</code> این عمل را انجام میدهد.
    مقادیر پارامترها برای این قسمت به صورت زیر انتخاب شده است. (تابع آماده از کتابخانه
      <span style="direction: rtl">OpenCV</span>
    انتخاب شده است.)
</p>


| Parameter | Value |
| - | - |
| winSize | 128, 128 |
| blockSize | (32, 32) |
| blockStride | (16, 16) |
| cellSize | (16, 16) |
| nbins | 9 |
| derivAperture | 1 |
| winSigma | -1 |.
| histogramNormType | 0 |
| L2HysThreshold | 0.2
| gammaCorrection | 1 |
| nlevels | 64 |
| signedGradient | True |


In [ ]:
def extract_HOG_features(self):

    gc.collect()

    self.hog_object = HOGFaceDetector.HOG(self.winSize)

    retriever1 = Retriever(self.train_hog_features_path)
    retriever2 = Retriever(self.validation_hog_features_path)
    retriever3 = Retriever(self.test_hog_features_path)

    try:
        self.train_HOG_features = retriever1.retrieve()
        self.validation_HOG_features = retriever2.retrieve()
        self.test_HOG_features = retriever3.retrieve()

        print('Retrieved from file ...')
    except:
        # for image in self.X_train:
        #     histogram_equalization(image)

        features_ = [self.hog_object.extract_features(image) for image in self.X_train]
        self.train_HOG_features = features_
        retriever1.save(features_)

        features_ = [self.hog_object.extract_features(image) for image in self.X_validation]

        self.validation_HOG_features = features_
        retriever2.save(features_)

        features_ = [self.hog_object.extract_features(image) for image in self.X_test]
        self.test_HOG_features = features_
        retriever3.save(features_)

    # clear space

    del self.X_train
    del self.X_validation
    del self.X_test
    gc.collect()
    time.sleep(5)

    logging.info('feature descriptors of train and test set extracted ... ')

    return self


***

### 3- Train SVM Model With Extracted Features Vectors

<p style="font-family: 'Vazir'; text-align: right; direction: rtl">
    برای آموزش مدل       <span style="direction: rtl">SVM</span> از توابع کتابخانه       <span style="direction: rtl">SKLearn</span>
    استفاده شده است. مقادیر هایپرپارامترها به صورت زیر است:
</p>

| Parameter | Optimum Value | Hypothesis Space |
| - | - | - |
| Kernle | Linear | Linear-RBF-Polynomial
| C | 0.01 |  $10^i : i = .0001, .001, .01, .1, 10$ |

<p style="font-family: 'Vazir'; text-align: right; direction: rtl">
    هسته خطی نسبت به بقیه هسته ها باعث میزان <span style="direction: rtl">Overfit</span> کمتری شده است.
</p>



In [ ]:
def classify_svm(self, kernel):

    self.scalar = StandardScaler()
    self.train_HOG_features = self.scalar.fit_transform(self.train_HOG_features)
    self.validation_HOG_features = self.scalar.transform(self.validation_HOG_features)
    self.test_HOG_features = self.scalar.transform(self.test_HOG_features)

    gc.collect()

    retriever = Retriever(self.svm_model_path)

    try:
        self.classifier_model = retriever.load()
        print('Retrieved from file ...')
    except:

        # construct classifier_model model
        svc = SVC(kernel=kernel,
                  C=0.01,
                  max_iter=-1,
                  probability=True,
                  cache_size=600, )

        # # estimate parameters
        # c_parameter = [i * 0.1 for i in range(3, 18)]
        # c_parameter = [1]
        # gamma_parameter = [10 ** i for i in range(-5, 1)]
        # gamma_parameter = [10 ** -4]
        #
        # parameters = [{'kernel': [kernel], 'C': c_parameter]
        #
        # grid_search = GridSearchCV(scoring='accuracy',
        #                            estimator=svc,
        #                            param_grid=parameters)
        #
        # # fit model to estimate
        # grid_search = grid_search.fit(self.validation_HOG_features, self.Y_validation)

        # train classifier
        self.classifier_model.fit(self.train_HOG_features,
                                  self.Y_train)

        retriever.save(self.classifier_model)

    # self.classifier_model = grid_search.best_estimator_

    # predict test image labels
    self.Y_test_predicted = self.classifier_model.predict(self.test_HOG_features)

    logging.info('test set classified ')

    self.compute_metrics()
    logging.info('metrics computed and saved ')

    # clear space
    del self.train_HOG_features
    del self.validation_HOG_features
    del self.test_HOG_features
    gc.collect()
    time.sleep(5)

    return self

<p style="font-family: 'Vazir'; text-align: right; direction: rtl">
    که نتایج حاصل شده به صورت زیر است:
</p>

| Parameter | Value |
| - | - |
| Accuracy | 0.998 |
| C | 0.01

![ROC Curve](res1.jpg)

![P-R Curve](res2.jpg)

<p style="font-family: 'Vazir'; text-align: right; direction: rtl">
 که تابع compute_metrics این عمل را انجام میدهد:
</p>

In [ ]:
def compute_metrics(self):
    y_score = self.classifier_model.decision_function(self.test_HOG_features)

    av_precision_recall_score = average_precision_score(self.Y_test,
                                                        y_score)

    logging.info('Average precision-recall score: {0:0.2f}'.format(av_precision_recall_score))

    display = plot_precision_recall_curve(self.classifier_model,
                                          self.test_HOG_features,
                                          self.Y_test)

    display.ax_.set_title('Precision Recall Curve: AP = ' + str(av_precision_recall_score))
    plt.savefig('res2.jpg', format='jpg', dpi=400)
    logging.info('Average Precision-Recall Score: {0:0.2f}'.format(av_precision_recall_score))

    roc_area_under_curve = roc_auc_score(self.Y_test,
                                         y_score)

    display = plot_roc_curve(self.classifier_model,
                             self.test_HOG_features,
                             self.Y_test)

    display.ax_.set_title('ROC Curve: AUC = ' + str(roc_area_under_curve))
    plt.savefig('res1.jpg', format='jpg', dpi=400)
    logging.info('ROC Area Under Curve: {0:0.2f}'.format(roc_area_under_curve))

***

### 4- Sliding Window:

<p style="font-family: 'Vazir'; text-align: right; direction: rtl">
 تابع <code style="">&#x200E;FaceDetector&#x200E;</code> این عمل را انجام میدهد.
 همچنین  برای Non Maximum Suppression از کتابخانه TensorFlow استفاده است.
 مورد دیگری که باید به آن توجه داشت این است که     <span style="direction: rtl">Sliding Window</span> در اسکیل های مختلف
 باید انجام شود.

</p>

In [ ]:
def FaceDetector(test_image, patch_h, patch_w, hog_face_detector: HOGFaceDetector):
    # equalized_histogram_test_image = histogram_equalization(test_image, copy=True)
    padding = 50
    equalized_histogram_test_image = add_padding(test_image, padding=padding)
    test_image = add_padding(test_image, padding=padding)

    hog: HOGFaceDetector.HOG = hog_face_detector.hog_object
    scalar: StandardScaler = hog_face_detector.scalar
    classifier_model: SVC = hog_face_detector.classifier_model

    equalized_histogram_test_image = cv2.cvtColor(equalized_histogram_test_image, cv2.COLOR_BGR2RGB)
    image_height, image_width, _ = equalized_histogram_test_image.shape

    scales = [.05 * i for i in range(8, 20)]

    import tensorflow as tf
    step = 1
    picked_patches = []
    score_thresh = .995
    for scale in scales:
        scaled_test_image = cv2.resize(equalized_histogram_test_image, dsize=(0, 0), fx=scale, fy=scale)
        H, W, _ = scaled_test_image.shape

        print('scale: ', str(scale))
        total_numbers = (H - patch_h) * (W - patch_w)

        patches = np.zeros((total_numbers, 6))
        index = 0
        for y in range(0, H - patch_h, step):
            for x in range(0, W - patch_w, step):
                part = scaled_test_image[y:y + patch_h, x:x + patch_w]
                feature_descriptors = hog.extract_features(part).reshape(1, -1)
                normal_feature_descriptors = scalar.transform(feature_descriptors)
                probability = classifier_model._predict_proba(normal_feature_descriptors)
                x1, y1, x2, y2 = x, y, x + patch_w, y + patch_h

                patches[index, :] = [probability[0, 1], y1, x1, y2, x2, scale]
                index += 1

        patches = patches[patches[:, 0] > score_thresh]
        picked_patches.append(patches)

    picked_patches = np.vstack(picked_patches)

    indices = tf.image.non_max_suppression(picked_patches[:, 1:5],
                                           picked_patches[:, 0],
                                           max_output_size=50,
                                           iou_threshold=.4,
                                           score_threshold=score_thresh).numpy()

    picked_patches = picked_patches[indices]

    indices = np.ones_like(indices)

    n = len(picked_patches)
    for i1 in range(n):
        for i2 in range(i1 - 1):
            p1 = picked_patches[i1]
            p2 = picked_patches[i2]
            y1_a, x1_a, y2_a, x2_a, scale_a = p1[1:]
            y1_a, x1_a, y2_a, x2_a = y1_a // scale_a, x1_a // scale_a, y2_a // scale_a, x2_a // scale_a

            y1_b, x1_b, y2_b, x2_b, scale_b = p2[1:]
            y1_b, x1_b, y2_b, x2_b = y1_b // scale_b, x1_b // scale_b, y2_b // scale_b, x2_b // scale_b

            S1 = (x2_a - x1_a) * (y2_a - y1_a)
            S1 = int(S1)

            S2 = (x2_b - x1_b) * (y2_b - y1_b)
            S2 = int(S2)

            overlap = max(0, min(x2_a, x2_b) - max(x1_a, x1_b)) * max(0, min(y2_a, y2_b) - max(y1_a, y1_b))
            overlap = int(overlap)

            union = S1 + S2 - overlap
            # union = overlap
            if union == S1:
                indices[i1] = 0

            elif union == S2:
                indices[i2] = 0

    indices = np.where(indices == 1)

    final_patches = picked_patches[indices]

    for index, patch in enumerate(final_patches):
        probability, y1, x1, y2, x2, scale = patch
        y, x, h, w = y1, x1, y2 - y1, x2 - x1
        y, x, h, w = y // scale, x // scale, h // scale, w // scale
        y, x, h, w = int(y), int(x), int(h), int(w),
        y, x = y, x
        y1, x1, y2, x2 = y, x, y + h, x + w
        color = (0, 255, 0)
        test_image = cv2.rectangle(test_image, (x1, y1), (x2, y2), color=color, thickness=2)

        color = (255, 255, 255)
        test_image = cv2.putText(test_image,
                                 'P: ' + str(round(probability, 3)) + ', Scale: ' + str(scale),
                                 (x1, y1 - 5),
                                 cv2.FONT_HERSHEY_PLAIN,
                                 .7,
                                 color,
                                 thickness=1,
                                 lineType=2)

    cv2.imshow('f', test_image)
    cv2.waitKey(0)

    return test_image


<hr style="height:3px;border:none;color:#333;background-color:#333;" />

### Results:

![Result 04](res4.jpg)

![Result 05](res5.jpg)

![Result 06](res6.jpg)

